In [1]:
!pip install underthesea


In [2]:
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from collections import Counter

from underthesea import word_tokenize

from keras import Input, Model
from keras.layers import Embedding, LSTM, Dense
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import RMSprop
import math
import warnings
warnings.filterwarnings("ignore")

In [3]:
df = pd.read_csv('chatbot3.csv', usecols=[0,1])
df

,question,answers
0,Làm thế nào mà thí nghiệm thành công?,chúng vẫn ấm và khô qua mùa đông
1,Khi nào phiên tiếp theo được công bố?,12 tháng 11 năm 1962
2,"Ai nói rằng Chopin đặt ra ""vào thế giới rộng l...",Zdzisław Jachimecki
3,Ai là quan chức cấp cao nhất trong Chính phủ N...,Thống đốc Chiết Giang
4,Oklahoma có bao nhiêu trường đại học công lập?,mười một
...,...,...
9993,Một nghiên cứu của ProPublica cho thấy một số ...,hơn 500.000 đô la
9994,Tên của con tàu đầu tiên sử dụng đèn của Ediso...,Columbia
9995,Ở những nước nào quặng uranium cao cấp đáng ch...,Canada
9996,Những trang trại sử dụng thuốc trừ sâu sinh họ...,trang trại độc canh


In [4]:
data_questions = df['question'].values
data_answers = df['answers'].values

In [5]:
#XỬ lý dữ liệu
def clean_text(sent):
    return re.sub(r'[!“”"#$%&\'()*+,-./:;<=>?@[\]^_`{|}~]', '', sent)

# hàm để chuyển Word Segmentation cho tiếng Việt
def clean_and_word_segmentation(sent):
    # Handle None or NaN values
    if sent is None or (isinstance(sent, float) and math.isnan(sent)):
        return ''
    # Convert to string
    sent = str(sent)
    # Check for rare data
    if len(sent) < 3:  # Adjust this threshold based on your data characteristics
        return ''
    # Clean and tokenize the text
    return word_tokenize(clean_text(sent.lower()))

In [6]:
count_words_ques = [len(clean_text(ques).split()) for ques in data_questions]
counter_words_ques = Counter(count_words_ques)

list_count_word = []
list_count_sent = []
for i in counter_words_ques.items():
    #print(i)
    list_count_word.append(i[0])
    list_count_sent.append(i[1])


In [7]:
len(count_words_ques), len(data_questions)


(9998, 9998)

In [41]:
# loại bỏ những câu có số lượng từ > 15 từ
sorted_ques = []
sorted_ans = []
for i,count in enumerate(count_words_ques):
    if count <= 15:
        sorted_ques.append(data_questions[i])
        sorted_ans.append(data_answers[i])

print('len sorted_ques:', len(sorted_ques))
print('len sorted_ans:', len(sorted_ans))
sorted_ques[:10], sorted_ans[:10]

len sorted_ques: 7182
len sorted_ans: 7182


(['Làm thế nào mà thí nghiệm thành công?',
  'Khi nào phiên tiếp theo được công bố?',
  'Ai là quan chức cấp cao nhất trong Chính phủ Nhân dân Chiết Giang?',
  'Oklahoma có bao nhiêu trường đại học công lập?',
  'Thí sinh nào đứng thứ ba về doanh số từ mùa ba?',
  'Hơn bất kỳ loài nào khác, chó có thể làm gì với con người?',
  'Hướng nào từ đảo là Đảo Chim nằm?',
  'Dự thảo kết thúc khi nào?',
  'Thời báo ủng hộ phe chính trị nào của Nội chiến Hoa Kỳ?',
  'Mà offshoot có ít điểm chung với chủ nghĩa lập thể?'],
 ['chúng vẫn ấm và khô qua mùa đông',
  '12 tháng 11 năm 1962',
  'Thống đốc Chiết Giang',
  'mười một',
  'William Hung',
  'hiểu và giao tiếp',
  'Southwest',
  '1973',
  'những người ly khai',
  'Futurist'])

In [42]:
# Clean and word segmentation for sorted_ques and sorted_ans
questions = [' '.join(clean_and_word_segmentation(ques)) for ques in sorted_ques]
answers = ['<START> ' + ' '.join(clean_and_word_segmentation(answ)) + ' <END>' for answ in sorted_ans]


print(questions[:10])
print(answers[:10])


['làm thế nào mà thí nghiệm thành công', 'khi nào phiên tiếp theo được công bố', 'ai là quan chức cấp cao nhất trong chính phủ nhân dân chiết giang', 'oklahoma có bao nhiêu trường đại học công lập', 'thí sinh nào đứng thứ ba về doanh số từ mùa ba', 'hơn bất kỳ loài nào khác chó có thể làm gì với con người', 'hướng nào từ đảo là đảo chim nằm', 'dự thảo kết thúc khi nào', 'thời báo ủng hộ phe chính trị nào của nội chiến hoa kỳ', 'mà offshoot có ít điểm chung với chủ nghĩa lập thể']
['<START> chúng vẫn ấm và khô qua mùa đông <END>', '<START> 12 tháng 11 năm 1962 <END>', '<START> thống đốc chiết giang <END>', '<START> mười một <END>', '<START> william hung <END>', '<START> hiểu và giao tiếp <END>', '<START> southwest <END>', '<START> 1973 <END>', '<START> những người ly khai <END>', '<START> futurist <END>']


In [11]:
# tokenize cho questions và answers
tokenizer = Tokenizer(filters='', lower=False)  # filters='' do dữ liệu đã được làm sạch và giữ lại word segmention
tokenizer.fit_on_texts(questions + answers)

# for word_index in tokenizer.word_index.items():
#     print(word_index)

In [12]:
VOCAB_SIZE = len(tokenizer.word_index) + 1
print(f'Vocabulary size : {VOCAB_SIZE}')

Vocabulary size : 8829


In [13]:
tokenized_questions = tokenizer.texts_to_sequences(questions)
maxlen_questions = 15
encoder_inp = pad_sequences(tokenized_questions,maxlen=maxlen_questions,padding='post')

print(encoder_inp.shape)
print(questions[0])
print(tokenized_questions[0])
print(encoder_inp[0])

(7182, 15)
làm thế nào mà thí nghiệm thành công
[26, 28, 3, 147, 1007, 461, 25, 27]
[  26   28    3  147 1007  461   25   27    0    0    0    0    0    0
    0]


In [14]:
# decoder
tokenized_answers = tokenizer.texts_to_sequences(answers)
maxlen_answers = np.max([len(x) for x in tokenized_answers])
decoder_inp = pad_sequences(tokenized_answers, maxlen=maxlen_answers, padding='post')

print(decoder_inp.shape)
print(answers[0])
print(tokenized_answers[0])
print(decoder_inp[0])

(7182, 45)
<START> chúng vẫn ấm và khô qua mùa đông <END>
[1, 533, 594, 2099, 13, 1472, 245, 211, 226, 2]
[   1  533  594 2099   13 1472  245  211  226    2    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0]


In [15]:
for i in range(len(tokenized_answers)):
    tokenized_answers[i] = tokenized_answers[i][1:]

padded_answers = pad_sequences(tokenized_answers, maxlen=maxlen_answers, padding='post')
decoder_final_output = to_categorical(padded_answers, VOCAB_SIZE)

print(decoder_final_output.shape)
print(tokenized_answers[0])
print(padded_answers[0])
print(decoder_final_output[0])

(7182, 45, 8829)
[533, 594, 2099, 13, 1472, 245, 211, 226, 2]
[ 533  594 2099   13 1472  245  211  226    2    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]]


In [16]:
enc_inputs = Input(shape=(None,))
enc_embedding = Embedding(VOCAB_SIZE, 200, mask_zero=True)(enc_inputs)
_, state_h, state_c = LSTM(200, return_state=True)(enc_embedding)
enc_states = [state_h, state_c]

dec_inputs = Input(shape=(None,))
dec_embedding = Embedding(VOCAB_SIZE, 200, mask_zero=True)(dec_inputs)
dec_lstm = LSTM(200, return_state=True, return_sequences=True)

dec_outputs, _, _ = dec_lstm(dec_embedding, initial_state=enc_states)
dec_dense = Dense(VOCAB_SIZE, activation='softmax')
output = dec_dense(dec_outputs)

In [17]:
model = Model([enc_inputs, dec_inputs], output)
model.compile(optimizer=RMSprop(), loss='categorical_crossentropy')
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, None)]               0         []                            
                                                                                                  
 input_2 (InputLayer)        [(None, None)]               0         []                            
                                                                                                  
 embedding (Embedding)       (None, None, 200)            1765800   ['input_1[0][0]']             
                                                                                                  
 embedding_1 (Embedding)     (None, None, 200)            1765800   ['input_2[0][0]']             
                                                                                              

In [19]:
from google.colab import drive
from keras.models import load_model

# Mount Google Drive
drive.mount('/content/drive')

# Đường dẫn tới file model trên Google Drive
model_path = '/content/drive/My Drive/chatbot.h5'

# Load model từ Google Drive
model = load_model(model_path)



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [20]:
def data_generator(encoder_inp, decoder_inp, decoder_final_output, batch_size):
    num_samples = len(encoder_inp)
    while True:
        for offset in range(0, num_samples, batch_size):
            encoder_batch = encoder_inp[offset:offset+batch_size]
            decoder_input_batch = decoder_inp[offset:offset+batch_size]
            decoder_output_batch = decoder_final_output[offset:offset+batch_size]
            yield [encoder_batch, decoder_input_batch], decoder_output_batch

batch_size = 64
model.fit(data_generator(encoder_inp, decoder_inp, decoder_final_output, batch_size),
          steps_per_epoch=len(encoder_inp) // batch_size,
          epochs=5)
model.save('chatbot.h5')



Epoch 1/5
112/112 [==============================] - 29s 214ms/step - loss: 0.0248
Epoch 2/5
112/112 [==============================] - 18s 160ms/step - loss: 0.0204
Epoch 3/5
112/112 [==============================] - 18s 164ms/step - loss: 0.0184
Epoch 4/5
112/112 [==============================] - 16s 143ms/step - loss: 0.0176
Epoch 5/5
112/112 [==============================] - 16s 143ms/step - loss: 0.0171


In [31]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, None)]               0         []                            
                                                                                                  
 input_2 (InputLayer)        [(None, None)]               0         []                            
                                                                                                  
 embedding (Embedding)       (None, None, 200)            1765800   ['input_1[0][0]']             
                                                                                                  
 embedding_1 (Embedding)     (None, None, 200)            1765800   ['input_2[0][0]']             
                                                                                              

In [32]:
from keras.models import Model
from keras.layers import Input, LSTM, Embedding, Dense

# Define the function to make inference models
def make_inference_models(model):
    # Extract the layers from the loaded model
    encoder_inputs = model.input[0]  # Encoder input
    decoder_inputs = model.input[1]  # Decoder input
    encoder_embedding = model.get_layer('embedding')(encoder_inputs)  # Encoder embedding
    decoder_embedding = model.get_layer('embedding_1')(decoder_inputs)  # Decoder embedding
    encoder_lstm = model.get_layer('lstm')
    encoder_outputs, state_h_enc, state_c_enc = encoder_lstm(encoder_embedding)  # Encoder LSTM and states
    decoder_lstm = model.get_layer('lstm_1')
    decoder_dense = model.get_layer('dense')

    # Create the encoder model
    encoder_model = Model(inputs=encoder_inputs, outputs=[state_h_enc, state_c_enc])

    # Create the decoder model
    decoder_state_input_h = Input(shape=(200,), name='input_3')
    decoder_state_input_c = Input(shape=(200,), name='input_4')
    decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

    decoder_outputs, state_h_dec, state_c_dec = decoder_lstm(
        decoder_embedding, initial_state=decoder_states_inputs)
    decoder_states = [state_h_dec, state_c_dec]
    decoder_outputs = decoder_dense(decoder_outputs)
    decoder_model = Model(
        inputs=[decoder_inputs] + decoder_states_inputs,
        outputs=[decoder_outputs] + decoder_states)

    print('Inference decoder:')
    decoder_model.summary()
    print('Inference encoder:')
    encoder_model.summary()
    return encoder_model, decoder_model

# Create inference models using the loaded model
enc_model, dec_model = make_inference_models(model)


Inference decoder:
Model: "model_6"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, None)]               0         []                            
                                                                                                  
 embedding_1 (Embedding)     (None, None, 200)            1765800   ['input_2[0][0]']             
                                                                                                  
 input_3 (InputLayer)        [(None, 200)]                0         []                            
                                                                                                  
 input_4 (InputLayer)        [(None, 200)]                0         []                            
                                                                         

In [33]:
def str_to_tokens(sentence):
    cleaned_and_segmented_sentence = clean_and_word_segmentation(sentence)
    if isinstance(cleaned_and_segmented_sentence, list):
        cleaned_and_segmented_sentence = ' '.join(cleaned_and_segmented_sentence)
    words = cleaned_and_segmented_sentence.lower().split()
    tokens_list = list()
    for current_word in words:
        result = tokenizer.word_index.get(current_word)
        if result is not None:
            tokens_list.append(result)
    return pad_sequences([tokens_list], maxlen=maxlen_questions, padding='post')


In [50]:
def chatbot():
    print('ChatBot: Xin chào!')

    while True:
        input_question = input('Question: ')

        if input_question == 'bye':
            print('ChatBot: bye')
            break
        states_values = enc_model.predict(str_to_tokens(input_question))
        empty_target_seq = np.zeros((1,1))
        empty_target_seq[0,0] = tokenizer.word_index['<START>']
        stop_condition = False
        decoded_translation = ''
        while not stop_condition:
            dec_outputs, h, c = dec_model.predict([empty_target_seq]+states_values)
            sampled_word_index = np.argmax(dec_outputs[0,-1, :])
            sampled_word = None
            for word, index in tokenizer.word_index.items():
                if sampled_word_index == index:
                    if word != '<END>':
                        decoded_translation += f'{word} '
                    sampled_word = word

            if sampled_word == '<END>' or len(decoded_translation.split()) > maxlen_answers:
                stop_condition = True
            empty_target_seq = np.zeros((1,1))
            empty_target_seq[0,0] = sampled_word_index
            states_values = [h,c]

        print('ChatBot :', decoded_translation, '\n')

In [43]:
ques_val = [' '.join(clean_and_word_segmentation(ques)) for ques in sorted_ques]
ans_val = [' '.join(clean_and_word_segmentation(answ)) for answ in sorted_ans]


print(ques_val[:3])
print(ans_val[:3])

['làm thế nào mà thí nghiệm thành công', 'khi nào phiên tiếp theo được công bố', 'ai là quan chức cấp cao nhất trong chính phủ nhân dân chiết giang']
['chúng vẫn ấm và khô qua mùa đông', '12 tháng 11 năm 1962', 'thống đốc chiết giang']


In [45]:
def calculate_accuracy(questions, answers, tokenizer, sample_size=None):
    total_correct = 0
    total_samples = len(questions)

    if sample_size and sample_size < total_samples:
        indices = np.random.choice(total_samples, sample_size, replace=False)
        questions = [questions[i] for i in indices]
        answers = [answers[i] for i in indices]
        total_samples = sample_size

    for i in range(total_samples):
        input_question = questions[i]
        true_answer = answers[i]

        # Chuyển đổi sang chuỗi nếu cần
        if isinstance(input_question, float):
            input_question = str(input_question)
        if isinstance(true_answer, float):
            true_answer = str(true_answer)

        # Dự đoán câu trả lời
        states_values = enc_model.predict(str_to_tokens(input_question))
        empty_target_seq = np.zeros((1, 1))
        empty_target_seq[0, 0] = tokenizer.word_index['<START>']
        stop_condition = False
        decoded_translation = ''
        while not stop_condition:
            dec_outputs, h, c = dec_model.predict([empty_target_seq] + states_values)
            sampled_word_index = np.argmax(dec_outputs[0, -1, :])
            sampled_word = None
            for word, index in tokenizer.word_index.items():
                if sampled_word_index == index:
                    if word != '<END>':
                        decoded_translation += f'{word} '
                    sampled_word = word

            if sampled_word == '<END>' or len(decoded_translation.split()) > maxlen_answers:
                stop_condition = True
            empty_target_seq = np.zeros((1, 1))
            empty_target_seq[0, 0] = sampled_word_index
            states_values = [h, c]

        # So sánh câu trả lời dự đoán với câu trả lời thực tế
        if decoded_translation.strip() == true_answer.strip():
            total_correct += 1

    accuracy = total_correct / total_samples
    return accuracy

# Đánh giá toàn bộ dữ liệu (có thể mất nhiều thời gian)
# accuracy = calculate_accuracy(sorted_ques, sorted_ans, tokenizer)
# print("Accuracy on full dataset:", accuracy)

# Hoặc đánh giá một phần dữ liệu để tiết kiệm thời gian
sample_size = 100  # Kiểm tra trên 100 cặp câu hỏi-câu trả lời
accuracy = calculate_accuracy(ques_val, ans_val, tokenizer, sample_size)
print(f"Accuracy on sample of {sample_size} data points:", accuracy)


1/1 [==============================] - 0s 19ms/step
Accuracy on sample of 100 data points: 0.92


In [51]:
chatbot()

ChatBot: Xin chào!
Question: Cảnh quan chính trị Bỉ bao gồm bao nhiêu thành phần?
1/1 [==============================] - 0s 29ms/step
ChatBot : hai thành phần  

Question: Tại sao Hegel tin rằng những thứ tự nhiên ít thực tế hơn những thứ thuộc linh?
1/1 [==============================] - 0s 19ms/step
ChatBot : chủ nghĩa chủng tộc và đại học tây bắc  

Question: Năm nào Olympic Air nhận được giải thưởng là Hãng hàng không nội địa hàng đầu?
1/1 [==============================] - 0s 19ms/step
ChatBot : phổ và valencian  

Question: Biệt thự nổi tiếng nhất ở Detroit là gì?
1/1 [==============================] - 0s 19ms/step
ChatBot : david  

Question: Flammeum là gì?
1/1 [==============================] - 0s 19ms/step
ChatBot : khăn choàng đỏ  

Question: bye
ChatBot: bye
